In [1]:
import tensorflow as tf
from tensorflow_privacy.privacy.optimizers.dp_optimizer_keras import DPKerasSGDOptimizer
from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy

# First impementation - with microbatching

In [2]:
# Model parameters
num_epochs = 15
batch_size = 256
learning_rate = 0.01

# DP parameters
noise_multiplier = 1.1
l2_norm_clip = 1.0
delta = 1e-5

In [3]:
num_train_samples = 60000  # MNIST training dataset size
num_steps = num_epochs * num_train_samples // batch_size

# Prepare the MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [4]:
# Build and compile your model as usual
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])

C:\Users\Noa\tensorflowPrivacy\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [5]:
# Use the DP version of an optimizer
optimizer = DPKerasSGDOptimizer(
    l2_norm_clip=l2_norm_clip,
    noise_multiplier=noise_multiplier,
    num_microbatches=batch_size,
    learning_rate=learning_rate
)


In [6]:

model.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=num_epochs, batch_size=batch_size)
# test_loss, test_accuracy = model.evaluate(y_test)

# Calculate the privacy budget
epsilon, optimal_order = compute_dp_sgd_privacy.compute_dp_sgd_privacy(
    n=num_train_samples,
    batch_size=batch_size,
    noise_multiplier=noise_multiplier,
    epochs=num_epochs,
    delta=delta
)

Epoch 1/15
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4637 - loss: 1.8265
Epoch 2/15
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8202 - loss: 0.8500
Epoch 3/15
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8505 - loss: 0.6121
Epoch 4/15
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8667 - loss: 0.5176
Epoch 5/15
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8790 - loss: 0.4591
Epoch 6/15
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8866 - loss: 0.4246
Epoch 7/15
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8899 - loss: 0.4036
Epoch 8/15
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8941 - loss: 0.3860
Epoch 9/15
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8959 - loss: 0.3703
Epoch 10/15
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9004 - loss: 0.3607
Epoch 11/15
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9035 - loss: 0.3449
Epoch 12/15
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step

In [7]:
print(f"DP-SGD with noise multiplier {noise_multiplier} and clip norm {l2_norm_clip} gives (ε = {epsilon:.2f}, δ = {delta}) for {num_epochs} epochs.")


DP-SGD with noise multiplier 1.1 and clip norm 1.0 gives (ε = 1.59, δ = 1e-05) for 15 epochs.


# SECOND METHOD 
without minibatches

In [9]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_privacy
from tensorflow_privacy.privacy.optimizers.dp_optimizer_keras import DPKerasSGDOptimizer
from tensorflow_privacy.privacy.optimizers.dp_optimizer_keras import DPKerasAdamOptimizer
from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy

In [10]:
# Load the MNIST dataset
(ds_train, ds_test), ds_info = tfds.load('mnist', split=['train', 'test'], shuffle_files=True, as_supervised=True, with_info=True)

In [11]:
# Preprocessing function to normalize the images and convert labels to categorical
def preprocess(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    label = tf.keras.utils.to_categorical(label, num_classes=10)
    return image, label

In [12]:
(ds_train_full, ds_test), ds_info = tfds.load('mnist', split=['train', 'test'], shuffle_files=True, as_supervised=True, with_info=True)

# Get the size of the datasets
full_train_size = ds_info.splits['train'].num_examples
test_size = ds_info.splits['test'].num_examples

# Calculate sizes for train and validation sets
train_size = int(0.8 * full_train_size)  # 80% for training
val_size = full_train_size - train_size  # 20% for validation

# Split the full training set into train and validation
ds_train = ds_train_full.take(train_size)
ds_val = ds_train_full.skip(train_size)

In [13]:
# # Apply preprocessing
ds_train = ds_train.map(preprocess).batch(train_size).prefetch(tf.data.experimental.AUTOTUNE)
ds_val = ds_val.map(preprocess).batch(val_size).prefetch(tf.data.experimental.AUTOTUNE)
ds_test = ds_test.map(preprocess).batch(test_size).prefetch(tf.data.experimental.AUTOTUNE)

In [14]:
# Define a simple fully connected model
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [15]:
learning_rate = 0.01
clipping_radius = 4
batch_number = 1
noise_multiplier = 0.8
epochs = 10

In [16]:
# Define the optimizer, loss, and metrics
optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
metrics = [tf.keras.metrics.CategoricalAccuracy()]

In [17]:
# Apply differentially private optimizer
dp_optimizer = DPKerasAdamOptimizer(
    l2_norm_clip=clipping_radius,
    noise_multiplier=noise_multiplier,
    num_microbatches=batch_number,  # No batching
    learning_rate=learning_rate
)

In [18]:
# Compile the model with the DP optimizer
# model.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss=loss, metrics=metrics)   #Adam version
model.compile(optimizer=dp_optimizer, loss=loss, metrics=metrics)    #dp version

# Train the model
history = model.fit(ds_train, epochs=epochs, validation_data=ds_val)  # is it ok?

Epoch 1/10


C:\Users\Noa\tensorflowPrivacy\Lib\site-packages\keras\src\backend\tensorflow\nn.py:567: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - categorical_accuracy: 0.0624 - loss: 2.4082 - val_categorical_accuracy: 0.4433 - val_loss: 1.7072
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 612ms/step - categorical_accuracy: 0.4456 - loss: 1.7031 - val_categorical_accuracy: 0.7122 - val_loss: 1.1802
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 568ms/step - categorical_accuracy: 0.7120 - loss: 1.1764 - val_categorical_accuracy: 0.6977 - val_loss: 0.9905
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 599ms/step - categorical_accuracy: 0.6942 - loss: 0.9886 - val_categorical_accuracy: 0.8033 - val_loss: 0.6762
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 581ms/step - categorical_accuracy: 0.8026 - loss: 0.6729 - val_categorical_accuracy: 0.8227 - val_loss: 0.5753
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 577ms/step - categorical_accuracy: 0.8244 - loss: 0.5716 - val_categorical_accuracy: 0.8341 - val_loss: 0.5427
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 576ms/step - categorical_accuracy: 0.8331 - loss: 0.5395 - val_categorical_

In [19]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(ds_test)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step - categorical_accuracy: 0.8844 - loss: 0.4037
Test Loss: 0.4036630094051361
Test Accuracy: 0.8844000101089478


In [22]:
# Compute the privacy budget

epsilon, best_alpha = compute_dp_sgd_privacy.compute_dp_sgd_privacy(
    n=train_size,  # number of training points
    batch_size=train_size,  # batch size
    noise_multiplier=noise_multiplier,  # noise multiplier
    epochs=epochs,  # number of epochs
    delta=1e-5  # delta
)
print(f"Epsilon: {epsilon}")
print(f"Optimal alpha: {best_alpha}")

# Test Loss: 2.3555104732513428
# Test Accuracy: 0.1551000028848648

DP-SGD with sampling rate = 100% and noise_multiplier = 0.8 iterated over 10 steps satisfies differential privacy with eps = 26.8 and delta = 1e-05.
The optimal RDP order is 2.25.
Epsilon: 26.78846537197618
Optimal alpha: 2.25


# increase number of layers

In [ ]:
# build the same way, with one one layer (at the same size) to ovserve the effect on train loss or accuracy, 